In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from pathlib import Path

%matplotlib inline

In [2]:
from elo_helpers import *

In [3]:
train = pd.read_feather('./data/train_v4.feather')
test = pd.read_feather('./data/test_v4.feather')

In [7]:
train.head()

,first_active_month,card_id,target,feature_1_mean_enc,feature_2_mean_enc,feature_3_mean_enc,feature_1_1,feature_1_2,feature_1_3,feature_1_4,...,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_std,new_installments_sum,new_installments_median,new_installments_max,new_installments_min,new_installments_std,new_month_lag_min,new_month_lag_max
0,6,C_ID_92a2005557,-0.820283,-0.501972,-0.342233,-0.429176,0,0,0,0,...,-0.296112,-0.724368,0.135812,0.0,0.0,0.0,0.0,0.000000,1.0,2.0
1,1,C_ID_3d0044924f,0.392913,-0.377079,-0.389225,-0.349962,0,0,0,1,...,-0.701858,-0.739410,0.014326,6.0,1.0,1.0,1.0,0.000000,1.0,2.0
2,8,C_ID_d639edf6cd,0.688056,-0.349808,-0.342233,-0.349962,0,1,0,0,...,-0.700326,-0.700326,NaN,0.0,0.0,0.0,0.0,NaN,2.0,2.0
3,9,C_ID_186d6a6901,0.142495,-0.377079,-0.512248,-0.349962,0,0,0,1,...,-0.566740,-0.734135,0.065882,5.0,1.0,1.0,-1.0,0.755929,1.0,2.0
4,11,C_ID_cdbd2c0db2,-0.159749,-0.305659,-0.512248,-0.349962,1,0,0,0,...,0.450886,-0.739395,0.223821,35.0,1.0,2.0,-1.0,0.376913,1.0,2.0


In [4]:
import xgboost as xgb
import lightgbm as lgb

In [5]:
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV, train_test_split

Overall strategy here will be to run the training set through a randomized search combined with KFolds for both XGBoost and LightGBM. Then, once I have the suggested parameters from the randomized search, repeat the process (maybe with KFold and maybe without) on a grid search of parameters that are close to those found by the randomized search. 

After the parameters are chosen, then I will submit predictions using each algorithm independently before trying mixes of the two algorithms' predictions (is this similar to regressing the two algo's preds onto the target?). 

## XGBoost

In [18]:
xgb_params = {
    'learning_rate': [0.03, 0.01, 0.005, 0.001],
    'max_depth': [5, 7, 10],
    'subsample': [0.6, 0.8, 0.95],
    'colsample_bytree': [0.6, 0.8, 0.9, 1],
    'colsample_bylevel': [0.75, 1],
    'n_estimators': [100, 175, 250],
    'min_child_weight': [2, 3, 6]
#     'objective': 'reg:linear', 
#     'eval_metric': 'rmse', 
#     'silent': True
}

In [19]:
X = train.drop(['card_id', 'target'], axis=1)
y = train.target

xgb_reg = xgb.XGBRegressor(n_jobs=-1, silent=True)

xgb_random = RandomizedSearchCV(xgb_reg,
                               param_distributions=xgb_params,
                               scoring= 'neg_mean_squared_error', #calc_rmse,
                               cv=5, n_jobs=-1)

In [20]:
xgb_random.fit(X, y)

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          fit_params=None, iid=True, n_iter=10, n_jobs=-1,
          param_distributions={'learning_rate': [0.03, 0.01, 0.005, 0.001], 'max_depth': [5, 7, 10], 'subsample': [0.6, 0.8, 0.95], 'colsample_bytree': [0.6, 0.8, 0.9, 1], 'colsample_bylevel': [0.75, 1], 'n_estimators': [100, 175, 250], 'min_child_weight': [2, 3, 6]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='neg_mean_squared_error',
          verbose=0)

In [24]:
xgb_random.best_params_

{'subsample': 0.8,
 'n_estimators': 175,
 'min_child_weight': 6,
 'max_depth': 5,
 'learning_rate': 0.03,
 'colsample_bytree': 0.8,
 'colsample_bylevel': 1}